# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
import numpy as np


# from skimage import data, measure
# from skimage.io import imread
# from skimage.filters import threshold_otsu
# from skimage.segmentation 

# from skimage.measure import regionprops
# from skimage.morphology import closing, square

# from skimage.color import label2rgb

import pickle

%matplotlib inline

# Utility function

In [ ]:
name_output_dir = 'run_256256Beta1_epochs30'
file_name_train='training_logs_'+name_output_dir+'.pkl'
file_name_valid='validation_logs_'+name_output_dir+'.pkl'


# file_name_train="training_logs_run_128128Beta1_epochs60.pkl"
# file_name_valid="validation_logs_128128Beta1_epochs60.pkl"

In [ ]:
def log_from_plk(file_name):
    file = open(file_name, "rb")
    output = pickle.load(file)
    #print(output_train)
    file.close()
    return output

## KL plot

In [ ]:
def Kl_logs(output_file):
    
    kl_per_levels = output_file["kls_loss_train"]
    kl_per_levels_mean = np.mean(kl_per_levels,axis=1)

    kl_0=[kl[0] for kl in kl_per_levels]
    kl_1=[kl[1] for kl in kl_per_levels]
    kl_2=[kl[2] for kl in kl_per_levels]
    kl_3=[kl[3] for kl in kl_per_levels]
    
    return kl_0,kl_1,kl_2,kl_3,kl_per_levels_mean

In [ ]:
def plot_mean_kl(kl_per_levels_mean):
    plt.figure(figsize=(20,10))
    plt.plot(kl_per_levels_mean[0],label='kl_mean_train')
    plt.plot(kl_per_levels_mean[1],label='kl_mean_valid')
    plt.xlabel("epochs")
    plt.ylabel("KL mean")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("Mean kl over during training")
    plt.savefig(os.path.join(r'C:\Users\youve\Dossier Thése\model_chkpoint\plot_metrics',name_output_dir+'kl_mean'+'.png'))

In [ ]:
def plot_kl(kl):
    
    plt.figure(figsize=(45,10))
    plt.subplot(1,4,1)
    plt.plot(kl[0][0],label='kl_0_train')
    plt.plot(kl[0][1],label='kl_0_valid')
    plt.xlabel("epochs")
    plt.ylabel("kl_0")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("KL level 0")
    
    
    plt.subplot(1,4,2)
    plt.plot(kl[1][0],label='kl_1_train')
    plt.plot(kl[1][1],label='kl_1_valid')
    plt.xlabel("epochs")
    plt.ylabel("kl_1")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("KL level 1")
    
    
    plt.subplot(1,4,3)
    plt.plot(kl[2][0],label='kl_2_train')
    plt.plot(kl[2][1],label='kl_2_valid')
    plt.xlabel("epochs")
    plt.ylabel("kl_2")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("KL level 2")
    
    
    
    plt.subplot(1,4,4)
    plt.plot(kl[3][0],label='kl_3_train')
    plt.plot(kl[3][1],label='kl_3_valid')
    plt.xlabel("epochs")
    plt.ylabel("kl_3")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("KL level 3")

    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
    plt.savefig(os.path.join(r'C:\Users\youve\Dossier Thése\model_chkpoint\plot_metrics',name_output_dir+'kl_losses'+'.png'))

In [ ]:
out_train_file,out_valid_file=log_from_plk(file_name_train),log_from_plk(file_name_valid)
out_train,out_valid=Kl_logs(out_train_file),Kl_logs(out_valid_file)
kl = [[kl_t,kl_v] for kl_t,kl_v in zip(out_train,out_valid)]
plot_kl(kl)

In [ ]:
plot_mean_kl(kl[4])

## ELBO

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(out_train_file["train_loss"],label="Train ELBO")
plt.plot(out_valid_file["val_loss"],label="Valid ELBO")
plt.xlabel("epochs")
plt.ylabel("ELBO")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
plt.title("ELBO during training")
plt.savefig(os.path.join(r'C:\Users\youve\Dossier Thése\model_chkpoint\plot_metrics',name_output_dir+'elbo'+'.png'))

##  Accuracy  metric (dice coef)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(out_train_file["dice_score_train"],label="Train DICE")
plt.plot(out_valid_file["dice_score_val"],label="Valid DICE")
plt.xlabel("epochs")
plt.ylabel("DICE")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
plt.title("DICE during training")
plt.savefig(os.path.join(r'C:\Users\youve\Dossier Thése\model_chkpoint\plot_metrics',name_output_dir+'dice'+'.png'))

## Reconstuction loss 

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(out_train_file["recons_loss_train"],label="Train Reconstruction")
plt.plot(out_valid_file["recons_loss_val"],label="Valid Reconstruction")
plt.xlabel("epochs")
plt.ylabel("Reconstruction")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
plt.title("Reconstruction during training")
plt.savefig(os.path.join(r'C:\Users\youve\Dossier Thése\model_chkpoint\plot_metrics',name_output_dir+'reconstruction_loss'+'.png'))


# Loss DeTr

In [ ]:
def losses_detr(out_file,mode='train'):
    keywords='detection_loss_train'
    if mode=='val':
        keywords='detection_loss_val'
    #print(keywords)
    detr_ce = [l[0] for l in out_file[keywords]]
    detr_bbox = [l[1] for l in out_file[keywords]]
    detr_giou = [l[2] for l in out_file[keywords]]
    
    return detr_ce,detr_bbox,detr_giou

In [ ]:
def plot_losses_detr(out_train_file,out_valid_file):
    
    detr_ce_t,detr_bbox_t,detr_giou_t=losses_detr(out_train_file)
    
    detr_ce_v,detr_bbox_v,detr_giou_v=losses_detr(out_valid_file,mode='val')
    
    
    plt.figure(figsize=(45,10))
    plt.subplot(1,3,1)
    plt.plot(detr_ce_t,label='CE_train')
    plt.plot(detr_ce_v,label='CE_valid')
    plt.xlabel("epochs")
    plt.ylabel("Cross Entropy")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("CE loss during training")
    
    
    plt.subplot(1,3,2)
    plt.plot(detr_bbox_t,label='bbox_train')
    plt.plot(detr_bbox_v,label='bbox_valid')
    plt.xlabel("epochs")
    plt.ylabel("bbox")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("bbox loss during training")
    
    
    plt.subplot(1,3,3)
    plt.plot(detr_giou_t,label='giou_train')
    plt.plot(detr_giou_v,label='giou_valid')
    plt.xlabel("epochs")
    plt.ylabel("kl_2")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), shadow=True, ncol=2)
    plt.title("giou loss during training")
    
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=None)
    plt.savefig('./plot_metrics/'+name_output_dir+'/detrlosses'+name_output_dir+'.png')
    
    

In [ ]:
plot_losses_detr(out_train_file,out_valid_file)

# load Test dataset ~10/20 patients